In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re
import warnings
warnings.filterwarnings(action='ignore')

In [11]:
filter_word = pd.read_excel('filter_word.xlsx')['filter_word'].values
path = 'new_output'

In [10]:
merge_file = dict()

merge_file['광진'] = ['광진리','광진','광진해변','휴휴암'] #광진 +강원도 삭제
merge_file['기사문'] = ['기사문리','기사문','기사문항','기사문해변']
merge_file['낙산'] = ['낙산','낙산항','낙산해수욕장']
merge_file['남애'] = ['남애리','남애','남애항'] #남애 +강원도 삭제
merge_file['동호'] = ['동호','동호리','동호해변','동호해수욕장'] #동호 +강원도 삭제 
merge_file['동산'] = ['동산','동산리','동산항','동산해수욕장'] #동산 +강원도 삭제
merge_file['물치'] = ['물치리','물치','물치항']
merge_file['수산'] = ['수산리','수산항'] #'수산' 제외
merge_file['오산'] = ['오산','오산해수욕장','오산리'] #오산 +강원도 삭제
merge_file['인구'] = ['인구리','죽도'] # '인구' 제외
merge_file['전진'] = ['전진리','설악해수욕장','후진항']
merge_file['하광정'] = ['하광정리','하조대','하조대해수욕장']

In [39]:
def separate_merge(path):
    '''파일분할되어 있는 결과를 종합해줌
    경로를 입력하면 파일들이 분할되어 있는 폴더를 선택하고
    폴더 내에 있는 파일들을 concat하여 폴더밖으로 배출
    '''
    file_list = os.listdir(f'./{path}/token/')
    folder_list = []
    #분할된 파일 탐색
    for file in file_list:
        if os.path.splitext(file)[1] != '.csv':
            folder_list.append(file)
            
    #분할된 파일 통합
    for folder in tqdm_notebook(folder_list):
        file_list = os.listdir(f'{path}/token/{folder}/')
        all_df = pd.DataFrame()
        for file in file_list:
            read_file = pd.read_csv(f'{path}/token/{folder}/{file}')
            all_df = pd.concat([all_df,read_file])
            size = all_df.shape[0]
        all_df.to_csv(f'./output/token/{folder}_0~{size}.csv',index=False)

In [42]:
def setting_file(path, merge_file):
    '''결합해야하는 파일들을 결합 이후 필터링 진행
    다양한 키워드를 하나의 keyword로 결합
    이후 다시한번 필터링 진행
    
    Parameter
    ---------
    path(string) : 파일이 있는 경로
    merge_file(dictionary) : 결합해야하는 keyword 
    
    Return 
    ------
    f'./output/token_통합/{key}_{total_len}.csv'의 형태로 파일저장
    '''
    os.makedirs(f'./{path}/token_통합',exist_ok=True)
    
    #대상 csv목록 생성
    file_list = os.listdir(f'./{path}/token')
    csv_list = []
    for file in file_list:
        if os.path.splitext(file)[1] == '.csv':
            csv_list.append(file)

    merge_file_values = list(itertools.chain(*merge_file.values()))

    #결합대상의 파일명을 가져오거나, 대상이 아닌 파일을 필터링
    files = dict()
    for file in csv_list:
        keyword = file.split('_')[0].split("+")[0].replace(' ', '')
        if keyword in merge_file_values:
            for key in merge_file.keys():
                if keyword in merge_file[key]:
                    if files.get(key) == None:
                        files[key] = [file]
                    else:
                        files[key].append(file)
        else :
            #필터링
            file_df = pd.read_csv(f'./{path}/token/{file}')
            file_df = file_df.fillna('')
            filtered = file_df.['full_text'].apply(lambda x : any(ele in x for ele in filter_word)) #전체글
            clean_file = file_df[~filtered]
            filtered = clean_file.title.apply(lambda x : any(ele in x for ele in filter_word)) #타이틀
            clean_file = clean_file[~filtered]
            
            total_len = clean_file.shape[0]
            clean_file.to_csv(f'./{path}/token_통합/{keyword}_{total_len}.csv',index = False)
            
    #파일결합
    for key in tqdm_notebook(files.keys()):
        merged_df = pd.DataFrame()
        for file in files[key]:
            file_df = pd.read_csv(f'./{path}/token/{file}')
            merged_df = pd.concat([merged_df,file_df])
        merged_df = merged_df.drop_duplicates('full_text')

        #필터링
        merged_df = merged_df.fillna('')
        filtered = merged_df['full_text'].apply(lambda x : any(ele in x for ele in filter_word)) #전체글
        clean_file = merged_df[~filtered]
        filtered = clean_file.title.apply(lambda x : any(ele in x for ele in filter_word)) #타이틀
        clean_file = clean_file[~filtered]

        total_len = clean_file.shape[0]
        clean_file.to_csv(f'./{path}/token_통합/{key}_{total_len}.csv',index = False)

In [43]:
separate_merge(path)
setting_file(path, merge_file)

### 파일 종합 결과

In [44]:
path = 'new_output'
def result_df(path):
    file_list = os.listdir(f'./{path}/token/')

    total_df = pd.DataFrame()
    for file in file_list:
        sumup = dict()
        keyword = file.split('_')[0]
        df = pd.read_csv(f'./{path}/token/{file}')
        sumup['total_size'] = df.shape[0]
        for year in range(2010,2020):
            try :
                sumup[year] = df[df.year == year].shape[0]
            except :
                sumup[year] = np.nan
        one_df = pd.DataFrame(sumup,index =[keyword])
        total_df = pd.concat([total_df,one_df])
    return total_df

In [45]:
result_df(path)

,total_size,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
강릉,6277,6277,0,0,0,0,0,0,0,0,0
